In [1]:
#@title Installing libraries { display-mode: "form" }
!pip install transformers 
!pip install tensorboard 
!pip install simpletransformers
!pip install jsonlines
!pip install pytelegrambotapi
!pip install --upgrade tqdm
!pip install pymorphy2[fast]
!pip install -U pymorphy2-dicts-ru

     |████████████████████████████████| 1.9MB 6.7MB/s 
     |████████████████████████████████| 890kB 25.8MB/s 
     |████████████████████████████████| 3.2MB 33.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=ab795266a688bc734a0521748cc7017c4dc3f6946bde9a2fdd763ec6155aa5af
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 215kB 5.2MB/s 
     |████████████████████████████████| 7.5MB 9.0MB/s 
     |████████████████████████████████| 81kB 12.2MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 317kB 48.8MB/s 
     |████████████████████████████████| 2.0MB 56.8MB/s 
     |████████████████████████████████| 1.2MB 58.1MB/s 
     |████████████████████████████████| 4.6MB 59.2MB/s 
     |████████████████████████████████| 112kB 53.9MB/s 
     |██████████████████████████████

     |████████████████████████████████| 81kB 4.4MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-3.7.6-cp37-none-any.whl size=59261 sha256=7960269af71e15ab356e9ac5eb64644320e0882821b453196c2a74553fa55fcc
  Stored in directory: /root/.cache/pip/wheels/f6/66/6e/d42e8fcb446d2683b5afe0b23318f4bb58896bad26549c47b9
Successfully built pytelegrambotapi
Requirement already up-to-date: tqdm in /usr/local/lib/python3.7/dist-packages (4.58.0)
     |████████████████████████████████| 61kB 4.0MB/s 
     |████████████████████████████████| 8.2MB 7.9MB/s 
     |████████████████████████████████| 378kB 48.4MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=847719 sha256=e766ada493ab337b6caed1a7d7c012587b928bca53c62aa67525da1705da2106
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG
Requirement already up-to-date: pymorphy2-dicts-ru in /usr/local/lib/python3.7/dist-packag

In [3]:
#@title Setup & Config & Imports
import pandas as pd
import numpy as np
import json
import jsonlines
from nltk.tokenize import word_tokenize
import re
from collections import defaultdict, Counter
import tqdm
class _TQDM(tqdm.tqdm):
    def __init__(self, *argv, **kwargs):
        kwargs['disable'] = True
        if kwargs.get('disable_override', 'def') != 'def':
            kwargs['disable'] = kwargs['disable_override']
        super().__init__(*argv, **kwargs)
tqdm.tqdm = _TQDM

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['text.color'] = mpl.rcParams['axes.labelcolor'] = mpl.rcParams['xtick.color'] = mpl.rcParams['ytick.color'] = 'white'

import seaborn as sns
%pylab inline
from sklearn.metrics import *

from simpletransformers.classification import ClassificationModel, ClassificationArgs

%load_ext tensorboard

from google.colab import files

from scipy.special import softmax
import nltk
nltk.download('punkt')


RS = 179
import random

Populating the interactive namespace from numpy and matplotlib
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
!rm -rf sample_data/

In [6]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Downloading data

In [7]:
!curl https://onti2020.ai-academy.ru/task/rucos.zip -o data.zip
!unzip -o data.zip

!mv RuCoS/ data/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 53.6M  100 53.6M    0     0  5415k      0  0:00:10  0:00:10 --:--:-- 6416k
Archive:  data.zip
   creating: RuCoS/
  inflating: __MACOSX/._RuCoS        
  inflating: RuCoS/.DS_Store         
  inflating: __MACOSX/RuCoS/._.DS_Store  
  inflating: RuCoS/rucos_test.jsonl  
  inflating: __MACOSX/RuCoS/._rucos_test.jsonl  
  inflating: RuCoS/rucos_val.jsonl   
  inflating: __MACOSX/RuCoS/._rucos_val.jsonl  
  inflating: RuCoS/rucos_train.jsonl  
  inflating: __MACOSX/RuCoS/._rucos_train.jsonl  


In [8]:
# model = torch.load('gdrive/MyDrive/model.bin')

# QA dataset class

In [109]:
import collections
import pymorphy2
import string

morph = pymorphy2.MorphAnalyzer(lang="ru")
words = dict()
parts = dict()
tt = str.maketrans(dict.fromkeys(string.punctuation))

def delete_punctuation(sentence):
    return sentence.translate(tt)

def delete_highlights(sentence):
    return re.sub(r'(@\w+)|(\\n)', " ", sentence)

def get_normal_form(word):
    if word in words:
        return words[word]
    was = word
    word = str(morph.parse(word)[0].normal_form)
    words[was] = word
    return word

def count_cnt_in(text, word):
    word = get_normal_form(word)

    return text.count(word)

def make_top_10(sentences, word):
    word = get_normal_form(word)
    d = collections.defaultdict(int)

    for now in sentences:
        if word not in now:
            continue
        for i in now:
            if i != word:
                if get_part(i) not in ['PREP', 'CONJ']:
                    d[i] += 1
    
    items = list(d.items())
    items.sort(key=lambda x: -x[1])
    return list(map(lambda x: x[0], items[:10]))
        

def get_part(word):
    if not word:
        return '-'
    if word in parts:
        return parts[word]
    was = word
    parts[was] = str(morph.parse(word)[0].tag.POS)
    if parts[was] == None:
        parts[was] = '-'
    return parts[was]

class QADataset: 
    """Question and answers dataset""" 
    def __init__(self, path, making_smaller=False, maxi_cnt=10000000): 
        random.seed(RS)
        """ 
        Args: 
            path (string): Path to jsonl file 
        """ 
        self.path = path 
        data = []
        bert_data = []
        self.text = [] 
        num = -1
        with open(path, 'r') as json_file: 
            json_list = list(json_file) 

            maxi1 = 0
            maxi2 = 0

            for json_str in tqdm.tqdm(json_list): 
                num += 1
                if num % 1000 == 0:
                    print(num)
                if num == maxi_cnt:
                    break
                item = json.loads(json_str) 
                
                text = item['passage']['text']
                
                entities = item['passage']['entities'] 
                
                used = list()
                for en in entities:
                    word = text[en['start']:en['end']] 
                    if word not in used:
                        used.append(word)
                entities = list(used)

                text = delete_highlights(text)

                self.text.append(text)

                sentences = [word_tokenize(delete_punctuation(i)) for i in text.split('.')]
                sentences = [list(map(lambda x: get_normal_form(x), sentence)) for sentence in sentences]
                text_without_punctuation = ' '.join(word_tokenize(delete_punctuation(text)))

                # catboost data
                for row in item['qas']: 
                    for ans in row.get('answers', [None]): 
                        q_text = row['query'] 
                        for i, word in enumerate(entities):
                            if ans:
                                label = int(ans['text'].strip() == word.strip())
                                tmp = random.randint(1, 10)
                                if not label and making_smaller and tmp != 1:
                                    continue

                            n_q = q_text.replace('@placeholder', word) 
                            top_10 = make_top_10(sentences, word)
                            top_10.extend(['-'] * (10 - len(top_10)))

                            data.append({"idx": f'{item["idx"]}/{i}', 
                                         "text_id": len(self.text) - 1})
                            bert_data.append({"idx": f'{item["idx"]}/{i}',
                                              "text_id": len(self.text) - 1,
                                              "query": n_q})
                            
                            for top in range(len(top_10)):
                                data[-1][f'top_{top + 1}'] = top_10[top]
                                data[-1][f'top_{top + 1}_part'] = get_part(top_10[top])
                            data[-1]['cnt_entity'] = count_cnt_in(text_without_punctuation, word)
                            data[-1]['entity_part'] = get_part(word)

                            if ans is not None:
                                bert_data[-1]["label"] = label
                                data[-1]["label"] = label
                    
        self.data = pd.DataFrame.from_dict(data).set_index("idx")
        self.bert_data = pd.DataFrame.from_dict(bert_data).set_index("idx")

    def get_bert_df(self):
        random.seed(RS)
        maxi1, maxi2 = 0, 0

        res = [] 
        rows = self.bert_data.iloc[:] 
        for idx, row in rows.iterrows(): 
            text_a = self.text[row.text_id].split() 
            text_b = row.query 

            w = 512 - len(text_b.split()) - 5 
            maxi1 = max(maxi1, len(text_a) + len(text_b.split()))
            e = random.randint(0, max(1, len(text_a) - w)) 
            text_a = ' '.join(text_a[e:e + w])
            maxi2 = max(maxi2, len(text_a.split()) + len(text_b.split()))

            res.append([idx, text_a, text_b, row.label if 'label' in self.data.columns else None]) 
        print(maxi1, maxi2)
        return pd.DataFrame(res, columns=["idx", "text_a", "text_b", "labels"]).set_index("idx")
        # return self.bert_data.drop('text_id', axis=1)

    def get_df(self):
        random.seed(RS)
        if 'text_id' in self.data.columns:
          return self.data.drop('text_id', axis=1)
        return self.data


In [110]:
%%time
train_df = QADataset('data/rucos_train.jsonl', making_smaller=True, maxi_cnt=7000)
val_df = QADataset('data/rucos_val.jsonl', making_smaller=True, maxi_cnt=100000)
test_df = QADataset('data/rucos_test.jsonl', making_smaller=False, maxi_cnt=100000)

0
1000
2000
3000
4000
5000
6000
7000
0
1000
2000
3000
4000
5000
6000
7000
0
1000
2000
3000
4000
5000
6000
7000
CPU times: user 1min, sys: 176 ms, total: 1min
Wall time: 1min


In [53]:
tmp = train_df.get_bert_df()

296 296


,text_a,text_b,labels
idx,,,
0/3,"Наблюдатели полагают, что подоплекой теракта в...","Кроме того, серьезным вызовом для России стано...",0
0/7,"Наблюдатели полагают, что подоплекой теракта в...","Кроме того, серьезным вызовом для России стано...",1
1/4,вторжении на Украину танковой колонны из РФ со...,Россия категорически опровергла сообщение Лысе...,1
1/7,вторжении на Украину танковой колонны из РФ со...,Россия категорически опровергла сообщение Киев...,0
1/8,О вторжении на Украину танковой колонны из РФ ...,Россия категорически опровергла сообщение Луга...,0
...,...,...,...
6999/3,"По словам Улюкаева, он думал, что в переданной...","По данным следствия, Игоря Сечина потребовал э...",0
6999/1,"словам Улюкаева, он думал, что в переданной ем...","По данным следствия, Сечина потребовал эти ден...",0
6999/2,"По словам Улюкаева, он думал, что в переданной...","По данным следствия, Улюкаев потребовал эти де...",1


In [70]:
data2281337 = val_df.get_df()

In [71]:
data2281337

,top_1,top_1_part,top_2,top_2_part,top_3,top_3_part,top_4,top_4_part,top_5,top_5_part,top_6,top_6_part,top_7,top_7_part,top_8,top_8_part,top_9,top_9_part,top_10,top_10_part,cnt_entity,entity_part,label
idx,,,,,,,,,,,,,,,,,,,,,,,
0/3,инициатива,NOUN,глава,NOUN,германский,ADJF,мид,NOUN,собрать,INFN,внешнеполитический,ADJF,ведомство,NOUN,страна,NOUN,нормандский,ADJF,формат,NOUN,0,NOUN,0
0/4,инициатива,NOUN,глава,NOUN,выступить,INFN,германский,ADJF,мид,NOUN,собрать,INFN,внешнеполитический,ADJF,ведомство,NOUN,страна,NOUN,нормандский,ADJF,0,NOUN,1
0/7,находиться,INFN,он,NPRO,приехать,INFN,составить,INFN,собственный,ADJF,впечатление,NOUN,ситуация,NOUN,восток,NOUN,украина,NOUN,мас,None,0,NOUN,0
0/4,инициатива,NOUN,глава,NOUN,выступить,INFN,германский,ADJF,мид,NOUN,собрать,INFN,внешнеполитический,ADJF,ведомство,NOUN,страна,NOUN,нормандский,ADJF,0,NOUN,1
0/14,германия,NOUN,сутки,NOUN,выявить,INFN,более,ADVB,100,None,новый,ADJF,заражение,NOUN,коронавирус,NOUN,тысяча,NOUN,демонстрант,NOUN,0,NOUN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7576/2,возбудить,INFN,дело,NOUN,министр,NOUN,здравоохранение,NOUN,саратовский,ADJF,область,NOUN,наталья,NOUN,уголовный,ADJF,другой,ADJF,сотрудник,NOUN,0,NOUN,1
7576/2,возбудить,INFN,дело,NOUN,министр,NOUN,здравоохранение,NOUN,саратовский,ADJF,область,NOUN,наталья,NOUN,уголовный,ADJF,другой,ADJF,сотрудник,NOUN,0,NOUN,1
7576/5,врачанестезиолог,NOUN,изнасиловать,INFN,пациентка,NOUN,операция,NOUN,скворцов,ADJS,поспорить,INFN,голиков,NOUN,«,None,ужасный,ADJF,»,None,0,NOUN,0


In [72]:
test_df.get_df()

,top_1,top_1_part,top_2,top_2_part,top_3,top_3_part,top_4,top_4_part,top_5,top_5_part,top_6,top_6_part,top_7,top_7_part,top_8,top_8_part,top_9,top_9_part,top_10,top_10_part,cnt_entity,entity_part
idx,,,,,,,,,,,,,,,,,,,,,,
0/0,год,NOUN,назад,ADVB,17,None,февраль,NOUN,2008,None,улица,NOUN,выйти,INFN,тысяча,NOUN,ликующий,ADJF,человек,NOUN,0,NOUN
0/1,год,NOUN,назад,ADVB,17,None,февраль,NOUN,независимость,NOUN,югославский,ADJF,2008,None,улица,NOUN,приштина,NOUN,выйти,INFN,0,NOUN
0/2,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,NOUN
0/3,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,NOUN
0/4,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,NOUN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7256/2,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,NOUN
7256/3,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,VERB
7256/4,президент,NOUN,сербия,NOUN,александр,NOUN,отменить,INFN,обращение,NOUN,нация,NOUN,поддержка,NOUN,экономика,NOUN,время,NOUN,эпидемия,NOUN,0,NOUN


# Modeling

In [73]:
!pip install catboost

In [74]:
catboost_train_df = val_df.get_df()
catboost_train_df

,top_1,top_1_part,top_2,top_2_part,top_3,top_3_part,top_4,top_4_part,top_5,top_5_part,top_6,top_6_part,top_7,top_7_part,top_8,top_8_part,top_9,top_9_part,top_10,top_10_part,cnt_entity,entity_part,label
idx,,,,,,,,,,,,,,,,,,,,,,,
0/3,инициатива,NOUN,глава,NOUN,германский,ADJF,мид,NOUN,собрать,INFN,внешнеполитический,ADJF,ведомство,NOUN,страна,NOUN,нормандский,ADJF,формат,NOUN,0,NOUN,0
0/4,инициатива,NOUN,глава,NOUN,выступить,INFN,германский,ADJF,мид,NOUN,собрать,INFN,внешнеполитический,ADJF,ведомство,NOUN,страна,NOUN,нормандский,ADJF,0,NOUN,1
0/7,находиться,INFN,он,NPRO,приехать,INFN,составить,INFN,собственный,ADJF,впечатление,NOUN,ситуация,NOUN,восток,NOUN,украина,NOUN,мас,None,0,NOUN,0
0/4,инициатива,NOUN,глава,NOUN,выступить,INFN,германский,ADJF,мид,NOUN,собрать,INFN,внешнеполитический,ADJF,ведомство,NOUN,страна,NOUN,нормандский,ADJF,0,NOUN,1
0/14,германия,NOUN,сутки,NOUN,выявить,INFN,более,ADVB,100,None,новый,ADJF,заражение,NOUN,коронавирус,NOUN,тысяча,NOUN,демонстрант,NOUN,0,NOUN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7576/2,возбудить,INFN,дело,NOUN,министр,NOUN,здравоохранение,NOUN,саратовский,ADJF,область,NOUN,наталья,NOUN,уголовный,ADJF,другой,ADJF,сотрудник,NOUN,0,NOUN,1
7576/2,возбудить,INFN,дело,NOUN,министр,NOUN,здравоохранение,NOUN,саратовский,ADJF,область,NOUN,наталья,NOUN,уголовный,ADJF,другой,ADJF,сотрудник,NOUN,0,NOUN,1
7576/5,врачанестезиолог,NOUN,изнасиловать,INFN,пациентка,NOUN,операция,NOUN,скворцов,ADJS,поспорить,INFN,голиков,NOUN,«,None,ужасный,ADJF,»,None,0,NOUN,0


In [75]:
catboost_test_df = test_df.get_df()
catboost_test_df

,top_1,top_1_part,top_2,top_2_part,top_3,top_3_part,top_4,top_4_part,top_5,top_5_part,top_6,top_6_part,top_7,top_7_part,top_8,top_8_part,top_9,top_9_part,top_10,top_10_part,cnt_entity,entity_part
idx,,,,,,,,,,,,,,,,,,,,,,
0/0,год,NOUN,назад,ADVB,17,None,февраль,NOUN,2008,None,улица,NOUN,выйти,INFN,тысяча,NOUN,ликующий,ADJF,человек,NOUN,0,NOUN
0/1,год,NOUN,назад,ADVB,17,None,февраль,NOUN,независимость,NOUN,югославский,ADJF,2008,None,улица,NOUN,приштина,NOUN,выйти,INFN,0,NOUN
0/2,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,NOUN
0/3,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,NOUN
0/4,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,NOUN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7256/2,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,NOUN
7256/3,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,VERB
7256/4,президент,NOUN,сербия,NOUN,александр,NOUN,отменить,INFN,обращение,NOUN,нация,NOUN,поддержка,NOUN,экономика,NOUN,время,NOUN,эпидемия,NOUN,0,NOUN


In [76]:
import pickle

train_pred_proba_misha_big = pickle.load(open('gdrive/MyDrive/out2/val_out_misha_small1.pickle', 'rb'))
test_pred_proba_misha_big = pickle.load(open('gdrive/MyDrive/out2/test_out_misha_small1.pickle', 'rb'))

# train_pred_proba_vova_big = pickle.load(open('gdrive/MyDrive/out2/val_out_vova_big_normal.pickle', 'rb'))
# test_pred_proba_vova_big = pickle.load(open('gdrive/MyDrive/out2/test_out_vova_big_normal.pickle', 'rb'))


In [77]:
len(val_df.get_df())

47935

In [78]:
len(train_pred_proba_misha_big) == len(val_df.get_df())

True

In [79]:
train_pred_proba_vova_big = train_pred_proba_vova_big[:len(catboost_train_df)]
train_pred_proba_misha_big = train_pred_proba_misha_big[:len(catboost_train_df)]

In [80]:
# catboost_train_df['model_vova_big'] = train_pred_proba_vova_big['proba']
# catboost_test_df['model_vova_big'] = test_pred_proba_vova_big['proba']

catboost_train_df['model_misha_small'] = train_pred_proba_misha_big['proba']
catboost_test_df['model_misha_small'] = test_pred_proba_misha_big['proba']

In [81]:
catboost_train_df

,top_1,top_1_part,top_2,top_2_part,top_3,top_3_part,top_4,top_4_part,top_5,top_5_part,top_6,top_6_part,top_7,top_7_part,top_8,top_8_part,top_9,top_9_part,top_10,top_10_part,cnt_entity,entity_part,label,model_misha_small
idx,,,,,,,,,,,,,,,,,,,,,,,,
0/3,инициатива,NOUN,глава,NOUN,германский,ADJF,мид,NOUN,собрать,INFN,внешнеполитический,ADJF,ведомство,NOUN,страна,NOUN,нормандский,ADJF,формат,NOUN,0,NOUN,0,0.002856
0/4,инициатива,NOUN,глава,NOUN,выступить,INFN,германский,ADJF,мид,NOUN,собрать,INFN,внешнеполитический,ADJF,ведомство,NOUN,страна,NOUN,нормандский,ADJF,0,NOUN,1,0.819388
0/7,находиться,INFN,он,NPRO,приехать,INFN,составить,INFN,собственный,ADJF,впечатление,NOUN,ситуация,NOUN,восток,NOUN,украина,NOUN,мас,None,0,NOUN,0,0.003573
0/4,инициатива,NOUN,глава,NOUN,выступить,INFN,германский,ADJF,мид,NOUN,собрать,INFN,внешнеполитический,ADJF,ведомство,NOUN,страна,NOUN,нормандский,ADJF,0,NOUN,1,0.819388
0/14,германия,NOUN,сутки,NOUN,выявить,INFN,более,ADVB,100,None,новый,ADJF,заражение,NOUN,коронавирус,NOUN,тысяча,NOUN,демонстрант,NOUN,0,NOUN,0,0.763440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7576/2,возбудить,INFN,дело,NOUN,министр,NOUN,здравоохранение,NOUN,саратовский,ADJF,область,NOUN,наталья,NOUN,уголовный,ADJF,другой,ADJF,сотрудник,NOUN,0,NOUN,1,0.014672
7576/2,возбудить,INFN,дело,NOUN,министр,NOUN,здравоохранение,NOUN,саратовский,ADJF,область,NOUN,наталья,NOUN,уголовный,ADJF,другой,ADJF,сотрудник,NOUN,0,NOUN,1,0.029479
7576/5,врачанестезиолог,NOUN,изнасиловать,INFN,пациентка,NOUN,операция,NOUN,скворцов,ADJS,поспорить,INFN,голиков,NOUN,«,None,ужасный,ADJF,»,None,0,NOUN,0,0.002742


In [82]:
import catboost

In [98]:
cnt = Counter(catboost_train_df['label'])
print(cnt)
clf = catboost.CatBoostClassifier(iterations=1000, task_type='GPU', class_weights=[1, cnt[0] / cnt[1]])

Counter({1: 24940, 0: 22995})


In [99]:
X_train = catboost_train_df.drop('label', axis=1)
y_train = catboost_train_df['label']

In [100]:
# X_val = val_df.drop('label', axis=1)
# y_val = val_df['label']

In [101]:
X_test = catboost_test_df

In [102]:
X_train

,top_1,top_1_part,top_2,top_2_part,top_3,top_3_part,top_4,top_4_part,top_5,top_5_part,top_6,top_6_part,top_7,top_7_part,top_8,top_8_part,top_9,top_9_part,top_10,top_10_part,cnt_entity,entity_part,model_misha_small
idx,,,,,,,,,,,,,,,,,,,,,,,
0/3,инициатива,NOUN,глава,NOUN,германский,ADJF,мид,NOUN,собрать,INFN,внешнеполитический,ADJF,ведомство,NOUN,страна,NOUN,нормандский,ADJF,формат,NOUN,0,NOUN,0.002856
0/4,инициатива,NOUN,глава,NOUN,выступить,INFN,германский,ADJF,мид,NOUN,собрать,INFN,внешнеполитический,ADJF,ведомство,NOUN,страна,NOUN,нормандский,ADJF,0,NOUN,0.819388
0/7,находиться,INFN,он,NPRO,приехать,INFN,составить,INFN,собственный,ADJF,впечатление,NOUN,ситуация,NOUN,восток,NOUN,украина,NOUN,мас,None,0,NOUN,0.003573
0/4,инициатива,NOUN,глава,NOUN,выступить,INFN,германский,ADJF,мид,NOUN,собрать,INFN,внешнеполитический,ADJF,ведомство,NOUN,страна,NOUN,нормандский,ADJF,0,NOUN,0.819388
0/14,германия,NOUN,сутки,NOUN,выявить,INFN,более,ADVB,100,None,новый,ADJF,заражение,NOUN,коронавирус,NOUN,тысяча,NOUN,демонстрант,NOUN,0,NOUN,0.763440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7576/2,возбудить,INFN,дело,NOUN,министр,NOUN,здравоохранение,NOUN,саратовский,ADJF,область,NOUN,наталья,NOUN,уголовный,ADJF,другой,ADJF,сотрудник,NOUN,0,NOUN,0.014672
7576/2,возбудить,INFN,дело,NOUN,министр,NOUN,здравоохранение,NOUN,саратовский,ADJF,область,NOUN,наталья,NOUN,уголовный,ADJF,другой,ADJF,сотрудник,NOUN,0,NOUN,0.029479
7576/5,врачанестезиолог,NOUN,изнасиловать,INFN,пациентка,NOUN,операция,NOUN,скворцов,ADJS,поспорить,INFN,голиков,NOUN,«,None,ужасный,ADJF,»,None,0,NOUN,0.002742


In [103]:
X_test

,top_1,top_1_part,top_2,top_2_part,top_3,top_3_part,top_4,top_4_part,top_5,top_5_part,top_6,top_6_part,top_7,top_7_part,top_8,top_8_part,top_9,top_9_part,top_10,top_10_part,cnt_entity,entity_part,model_misha_small
idx,,,,,,,,,,,,,,,,,,,,,,,
0/0,год,NOUN,назад,ADVB,17,None,февраль,NOUN,2008,None,улица,NOUN,выйти,INFN,тысяча,NOUN,ликующий,ADJF,человек,NOUN,0,NOUN,0.063657
0/1,год,NOUN,назад,ADVB,17,None,февраль,NOUN,независимость,NOUN,югославский,ADJF,2008,None,улица,NOUN,приштина,NOUN,выйти,INFN,0,NOUN,0.224905
0/2,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,NOUN,0.006426
0/3,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,NOUN,0.070049
0/4,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,NOUN,0.002487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7256/2,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,NOUN,0.006864
7256/3,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,-,None,0,VERB,0.815820
7256/4,президент,NOUN,сербия,NOUN,александр,NOUN,отменить,INFN,обращение,NOUN,нация,NOUN,поддержка,NOUN,экономика,NOUN,время,NOUN,эпидемия,NOUN,0,NOUN,0.863392


In [104]:
cat_features = [f'top_{i}_part' for i in range(1, 11)] + [f'top_{i}' for i in range(1, 11)] + ["entity_part", ]
# cat_features = ["entity_part", ]
data_for_catboost = catboost.Pool(X_train, y_train, cat_features=cat_features)

In [ ]:
clf.fit(data_for_catboost, plot=True)

# Make submition

In [106]:
actual_test = test_df

def save_pred(filename, preds):
    with open('data/rucos_test.jsonl', 'r') as json_file:
        test = list(map(lambda x: json.loads(x), list(json_file)))
    ind = 0
    res = []
    
    for i in range(len(test)):
        ent_prob = []
        
        entities = test[i]['passage']['entities']
        used = set()
        for en in entities:
            word = test[i]['passage']['text'][en['start']:en['end']] 
            if word not in used:
                used.add(word)
        entities = list(used)
        
        for j in range(len(entities)):
            ent_prob.append(preds[ind])
            ind += 1
        ent_prob = np.array(ent_prob).argmax()
        
#         start, end = test[i]['passage']["entities"][ent_prob]['start'], test[i]['passage']["entities"][ent_prob]['end']
        answer = entities[ent_prob]
        res.append({"idx": test[i]["idx"], "text": answer})
    
    !mkdir submitions/
    
    with jsonlines.open("submitions/" + filename, 'w') as file:
        file.write_all(res)

In [107]:
predictions = clf.predict_proba(X_test)[:,1]

In [108]:
predictions

array([0.14512313, 0.23188649, 0.0759114 , ..., 0.45483931, 0.15351739,
       0.01316424])

In [111]:
cnt = Counter(clf.predict(X_test))

print(f'    cnt: {cnt}')
print(f"0 label: {cnt[0] / len(predictions) * 100:.3f}%")
print(f"1 label: {cnt[1] / len(predictions) * 100:.3f}%")

    cnt: Counter({0: 55494, 1: 12257})
0 label: 81.909%
1 label: 18.091%


In [112]:
file_name = f'{__import__("datetime").datetime.now()}.jsonl'
save_pred(file_name, test_pred_proba_misha_big['proba'])

mkdir: cannot create directory ‘submitions/’: File exists


In [113]:
files.download('submitions/' + file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Telegram Logger (doc sender) { run: "auto", vertical-output: true, display-mode: "form" }
token = "1282076787:AAEC3rYsSHh_livmutl0qDEemVhlhjWpWyI" #@param {type:"string"}
user_id =  -584355986 #@param {type:"integer"}
combine = True #@param ["True", "False"] {type:"raw"}

import telebot
bot = telebot.TeleBot(token)
last_message = None

def logger(txt):
    global last_message
    while True:
        try:
            last_message_text = last_message.text if last_message != None else ''

            if type(txt) is tqdm:
                txt = txt.format_dict

                rate = txt["rate"] if txt["rate"] else 0
                remain = int((txt['total'] - txt['n']) / rate) if rate != 0 else 0

                txt = f"{txt['prefix']} - {int(txt['n'] / txt['total'] * 100)}%, {txt['n']}/{txt['total']} [{str(int(txt['elapsed']) // 60).rjust(2, '0')}:{str(int(txt['elapsed']) % 60).rjust(2, '0')}<{str(remain // 60).rjust(2, '0')}:{str(remain % 60).rjust(2, '0')}]"
                
                last_message_text = '\n'.join(last_message_text.split('\n')[:-1])

            if not last_message or not combine:
                last_message = bot.send_message(user_id, txt)
            else:
                last_message = bot.edit_message_text(chat_id=user_id,
                                      message_id=last_message.message_id,
                                      text=last_message_text + '\n' + txt)
            return
        except Exception as ex:
            print("Exception while logging:")
            print(ex)
            pass

In [ ]:
def eval_info(model, predictions, y_test):
    from sklearn.metrics import f1_score
    print(model)
    # print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro', zero_division=0)))
    # print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro', zero_division=0)))
    print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro', zero_division=0)))
    # print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))
    # print(classification_report(y_test, predictions, zero_division=0))
    # labels = ['0', '1']
    # sns.heatmap(data=confusion_matrix(y_test, predictions), annot=True, fmt="d", cbar=False, xticklabels=labels, yticklabels=labels)
    # plt.title("Confusion matrix")
    # plt.show()

In [ ]:
eval_info(clf, clf.predict(X_val), y_val)
f1 = f1_score(y_val, clf.predict(X_val), average='macro', zero_division=0)

F1-measure:   0.40


In [ ]:
# while True:
#     try:
#         bot.send_document(user_id, 
#                           open('submitions/' + file_name, 'rb'),
#                           caption=f'{cnt}\nF1 score on val set: {f1 * 100:.2f}%\nLabel 0: {cnt[0] / len(predictions) * 100:.3f}%\nLabel 1: {cnt[1] / len(predictions) * 100:.3f}%')
#         break
#     except :
#         import time
#         time.sleep(1)

### Save tensorboard

In [ ]:
# !zip -r runs.zip runs/

In [ ]:
# files.download('runs.zip')

In [ ]:
# while True:
#     try:
#         bot.send_document(user_id, open('runs.zip', 'rb'), caption=f'{cnt}\nF1 score on val set: {f1 * 100:.2f}%')
#         break
#     except:
#         import time
#         time.sleep(1)

In [ ]:
# while True:
#     try:
#         bot.send_document(user_id, open('outputs/training_progress_scores.csv', 'rb'), caption=f'{cnt}\nF1 score on val set: {f1 * 100:.2f}%')
#         break
#     except:
#         import time
#         time.sleep(1)

In [ ]:
# !zip -r model.zip outputs/best_model/

In [ ]:
# files.download('model.zip')

In [ ]:
# cnt = 0
# while cnt < 5:
#     try:
#         with open('model.zip', 'rb') as file:
#             bot.send_document(user_id, file, caption=f'{cnt}\nF1 score on val set: {f1 * 100:.2f}%')
#             break
#     except Exception as ex:
#         print(ex)
#         cnt += 1
#         import time
#         time.sleep(1)

In [ ]:
# !rm -rf outputs/